# Classifier to NER

This notebook will first used a model to automate the classification of data and then apply the corresponding NER model to optimize the retrieval process.

## specific case

In [ ]:
import spacy
import pandas as pd
import json
import os

In [ ]:
ROOT_PATH = r'..'

Class...

In [ ]:
# Load the model
nlp = spacy.load(os.path.join(ROOT_PATH, r"model\classifyer\PPV\model-best"))

# Load the data
data = pd.read_csv(os.path.join(ROOT_PATH, r"\data\training_csv\data449_cleaned.csv"))

In [ ]:
texts = data["text"].tolist()

for text in texts:
    doc = nlp(text)
    print(doc.cats,  "-",  text)

In [ ]:
ppv_list = []
nppv_list = []

for text in texts:
    doc = nlp(text)
    if doc.cats.get("PPV", 0.0) > doc.cats.get("NPPV", 0.0):
        ppv_list.append(text)
    else:
        nppv_list.append(text)

...to NER

In [ ]:
nlp_ner = spacy.load(os.path.join(ROOT_PATH, r"model\unilabel\PPV_V2\model-best"))  # Select another model to test it

# Initialize an empty list to store the output data
output_data = []

# Loop through items in the directory
for item in ppv_list:
    # with open(item, "r", encoding="utf-8") as file:
    #     text = file.read()

    doc = nlp_ner(text)
    
    # Extract labeled information
    labeled_data = [
        {
            "start": ent.start_char,
            "end": ent.end_char,
            "labels": ent.label_
        }
        for ent in doc.ents
    ]

    # Store text and labeled_data as a tuple and append to the output_data list
    output_data.append((text, {"label": labeled_data}))

# Save the labeled data to a JSON file
with open(os.path.join(ROOT_PATH, r"data\predicted_json\labeled_data_PPV_class_to_NER.json"), "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## General case

In [ ]:
import spacy
import pandas as pd
import json

def classify(classifier_model, data):
    """
    This function takes in text stored in csv file and outputs two lists of text for the different labels.
    
    args:
        classifier_model: the path to the model used to classify the text
        data: the path to the csv file containing the text
    """

    # Load the model and data
    nlp_classify = spacy.load(classifier_model)
    data = pd.read_csv(data)

    # Extract the text from the data and class them
    texts = data["text"].tolist()

    for text in texts:
        doc = nlp_classify(text)
        # print(doc.cats,  "-",  text)

    # Store them in corresponding lists
    ppv_list = []
    nppv_list = []

    for text in texts:
        doc = nlp_classify(text)
        if doc.cats.get("PPV", 0.0) > doc.cats.get("NPPV", 0.0):
            ppv_list.append(text)
        else:
            nppv_list.append(text)
    return ppv_list, nppv_list

def NER(ner_model, list:list):
    """
    This function takes in a list of text and outputs a list of tuples containing the text and the predicted NER labels.

    args:
        ner_model: the path to the model used to predict the NER labels
        list: the list of text to be predicted
    """
    nlp_ner = spacy.load(ner_model)

    # Initialize an empty list to store the output data
    output_data = []

    # Loop through items in the directory
    for text in list:
        doc = nlp_ner(text)
        
        # Extract labeled information
        labeled_data = [
            {
                "start": ent.start_char,
                "end": ent.end_char,
                "labels": ent.label_
            }
            for ent in doc.ents
        ]

        # Store text and labeled_data as a tuple and append to the output_data list
        output_data.append((text, {"label": labeled_data}))
    return output_data

def open_json(json_file:str):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def save_NER_to_json(list:list, output_path:str):
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(list, json_file, ensure_ascii=False, indent=4)
    
def json_file_length(json_file:str):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    lenght = len(data)
    print("Le dossier JSON contient", lenght, "éléments.")

def NER_pipeline(classifier_model, ner_model, data_input, output_path):
    """
    This function takes a classifier model, data input, and an output file path.
    It classifies the text using the classifier model and then applies the NER model
    to the classified text, returning a JSON file of predicted NER labels.

    Args:
        classifier_model: The path to the model used to classify the text.
        ner_model: The path to the model used to predict the NER labels.
        data_input: The path to the CSV file containing the text.
        output_path: The path to the JSON file to store the predicted NER labels.
    """
    # Use the classify function to classify the data
    ppv_list, nppv_list = classify(classifier_model, data_input)

    # Use the NER function to predict the NER labels
    ppv_output = NER(ner_model, ppv_list)

    # Save the predicted NER labels to a JSON file
    save_NER_to_json(ppv_output, output_path)

    # Print the length of the JSON file
    json_file_length(output_path)

In [ ]:
classifier_mode = os.path.join(ROOT_PATH, r"model\classifyer\PPV\model-best")
ner_model = os.path.join(ROOT_PATH, r"model\unilabel\PPV_V2\model-best")
data_input = os.path.join(ROOT_PATH, r"data\training_csv\data449_cleaned.csv")
output_path = os.path.join(ROOT_PATH, r"data\predicted_json\labelled_data_PPV_class_to_NER.json")

NER_pipeline(classifier_model=classifier_mode, ner_model=ner_model, data_input=data_input, output_path=output_path)

Evaluate the predictions

In [ ]:
import sys
import seaborn as sns
sys.path.append(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\script")

from confusion_matrix_spacy_def import automate_confusion_matrix

In [ ]:
# Define the target labels and the path to the JSON files
target_labels = ['PPV']
true_json_file = os.path.join(ROOT_PATH, r"data\training_json\data449.json")
pred_json_file = os.path.join(ROOT_PATH, r"data\predicted_json\labelled_data_PPV_class_to_NER.json")

# Generate the confusion matrix
automate_confusion_matrix(true_json_file, pred_json_file, target_labels)